In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from utils.objects import *
from utils.main_objects import *

## Importing the data

In [2]:
rsu = pd.read_csv('data/rsu.csv')
es = pd.read_csv('data/es.csv')
tasks = pd.read_csv('data/tasks.csv')

In [3]:
rsu_bank = populate_rsus(rsu)
es_bank = populate_ess(es)
tasks_bank = populate_tasks(tasks)

In [4]:
len(rsu_bank), len(es_bank), len(tasks_bank)

(100, 100, 50)

## Generating a network

In [5]:
# we repopulate the rsu and es banks to ensure the network is new
rsu_bank = populate_rsus(rsu)
es_bank = populate_ess(es)
network = get_network(rsu_bank, es_bank)

In [6]:
M, W

(95, 1)

In [7]:
# put the network in a dataframe with the attributes of the objects as columns
# network is a list of objects
network_df = pd.DataFrame([vars(x) for x in network])
network_df.head()

,ID,X,Y,DTR,STATE,END_TIME,ES
0,28,45,47,10,IDLE,None,AP
1,91,1,0,3,IDLE,None,AP
2,97,37,83,46,IDLE,None,AP
3,61,49,76,58,IDLE,None,AP
4,27,87,58,21,IDLE,None,AP
...,...,...,...,...,...,...,...
90,19,57,53,13,IDLE,None,AP
91,89,63,66,19,IDLE,None,AP
92,95,73,66,11,IDLE,None,AP
93,34,84,89,18,IDLE,None,AP


In [8]:
tasks_df = pd.DataFrame([vars(x) for x in tasks_bank])
tasks_df.head()

,ID,LENGTH,FILE_SIZE,TYPE,X,Y,COMPLETED,MIGRATION_HISTORY,COMPUTATION_HISTORY,MIGRATION_TIME,RSU_HISTORY
0,0,32,278,COMPUTATION,30,74,False,0,0,0,[]
1,1,74,175,DATA TRANSFER,36,93,False,0,0,0,[]
2,2,60,400,COMPUTATION,14,51,False,0,0,0,[]
3,3,68,251,DATA TRANSFER,46,69,False,0,0,0,[]
4,4,91,468,COMPUTATION,36,89,False,0,0,0,[]
5,5,1,428,DATA TRANSFER,31,56,False,0,0,0,[]
6,6,86,493,COMPUTATION,62,71,False,0,0,0,[]
7,7,98,360,COMPUTATION,5,19,False,0,0,0,[]
8,8,33,228,COMPUTATION,14,10,False,0,0,0,[]
9,9,23,333,COMPUTATION,48,37,False,0,0,0,[]


## Running the task offloading process

In [9]:
current_time = 0

# while there are still tasks to be completed
while not all(task.COMPLETED for task in tasks_bank):

    # get the first task in the task bank which is not completed
    task = next((task for task in tasks_bank if task.COMPLETED == False), None)

        # shuffle the tasks bank
        np.random.shuffle(tasks_bank)
        task = next(task for task in tasks_bank if not task.COMPLETED and task.MIGRATION_TIME == 0)

        # if the task was not being migrated, fin the closest rsu to the task
        if task.RSU_HISTORY == []:
        # get the closest rsu to the task
            closest_rsu = get_closest_rsu(network, task, task.RSU_HISTORY)

            task.RSU_HISTORY.append(closest_rsu)

            # suppose the task is assigned at that rsu directly
            task.X = closest_rsu.X
            task.Y = closest_rsu.Y
        else:
            # get the last rsu the task was assigned to
            closest_rsu = task.RSU_HISTORY[-1]

        # if the rsu is free and compatible with the task, compute the task
        if is_server_free(closest_rsu) and compatible(closest_rsu, task):
            closest_rsu.STATE = "BUSY"
            task.COMPLETED = True
            # if the task is of type 'COMPUTATION', compute the computation time
            if task.TYPE == "COMPUTATION":
                t_computation = computation_time(task_length=task.LENGTH,
                                                 vm_nb=closest_rsu.ES.VM_NB,
                                                 vm_cpu=closest_rsu.ES.VM_CP)
                closest_rsu.END_TIME = current_time + t_computation

                task.COMPUTATION_HISTORY = t_computation
            # else (if the task is of type 'DATA TRANSFER'), compute the transfer time
            elif task.TYPE == "DATA TRANSFER":
                t_data_transfer = data_transfer_time(file_size=task.FILE_SIZE, 
                                                     dtr=closest_rsu.DTR)
                closest_rsu.END_TIME = current_time + t_data_transfer
                task.COMPUTATION_HISTORY = t_data_transfer

        # if the rsu is busy or incompatible with the task, migrate the task
        else:
            # find the closest rsu to the task
            closest_rsu = get_closest_rsu(network, task, task.RSU_HISTORY)

            # add the closest rsu to the history of the task
            task.RSU_HISTORY.append(closest_rsu)
            
            # compute the migration time
            t_migration = migration_time(file_size=task.FILE_SIZE,
                                            distance=distance(task, closest_rsu),
                                            dtr=closest_rsu.DTR)
            
            # update the migration time of the task
            task.MIGRATION_TIME = t_migration
            # keep a record of the total migration time of the task
            task.MIGRATION_HISTORY += t_migration

            # update the coordinates of the task for when it will be migrated
            task.X = closest_rsu.X
            task.Y = closest_rsu.Y

    # check if any rsu should turn free at the current time
    for rsu in rsu_bank:
        if rsu.STATE == "BUSY" and rsu.END_TIME == current_time:
                rsu.STATE = "IDLE"
                rsu.END_TIME = 0

    # remove a time unit from the migration time of each task if the migration time is greater than 0
    for task in tasks_bank:
        if task.MIGRATION_TIME > 0:
            task.MIGRATION_TIME -= 1

    # if a task has gone through all the RSU, reset its history
    for task in tasks_bank:
        if len(task.RSU_HISTORY) == len(network):
            task.RSU_HISTORY = []

    # update the current time
    current_time += 1



  0%|          | 0/50 [00:00<?, ?it/s]